# 인스타그램 크롤링
이번 방법은 해시태그가 아닌 정보공유성 계정 크롤링을 진행한다.

모든 피드를 가져와 브랜드 및 주소를 추출하기 전에, 상대적으로 추출하기 쉬운 데이터로 도전

추후 감성분석 및 빅데이터 구성에는 해시태그 또한 추출이 필요할 것으로 보임

## Library Import

In [1]:
import pandas as pd
import re #정규식

from tqdm import tqdm

from PyKakao import Local

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import requests
from bs4 import BeautifulSoup

import time

# 데이터 정제용
import re
import emoji
from soynlp.normalizer import repeat_normalize

## Crawling

### 크롬드라이버 설치

In [4]:
service = Service(ChromeDriverManager().install()) 

### 옵션 설정

In [5]:
# User Agent 설정
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'

options = Options()

# options.add_argument('--proxy-server=socks5://127.0.0.1:9050') #proxy 우회 방법, Instagram에서는 사용할 수 없음(차단당함)
options.add_argument('--start-maximized')
# options.add_argument('--incognito') #시크릿모드
# options.add_argument('--headless') #화면 Open되지 않음

options.add_experimental_option('excludeSwitches', ['enable-automation']) #'자동화된 소프트웨어에 의해 제어되고 있습니다' 해제

### 크롬 Open
따로 Cell 분리해둔 이유는 IP우회를 위해 플러그인 세팅을 해줘야하기 때문임

In [24]:
driver = webdriver.Chrome(service=service, options=options)

### 필요 함수 선언
#### login : 로그인 및 불필요 팝업안내 제거
#### search : 검색어로 검색하여 포스트 링크를 가져옴
#### get_post : 링크로 들어가 게시물과 게시시간을 가져옴

In [27]:
def login(id, pw):
    
    # 로그인 페이지로 이동
    driver.get('https://www.instagram.com/accounts/login/')
    
    # id와 pw를 입력하는 창의 요소 정보 획득
    WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.TAG_NAME, 'input')))
    time.sleep(1)
    
    input = driver.find_elements(By.TAG_NAME, 'input')

    # 아이디를 입력
    input[0].send_keys(id)

    # 비밀번호 입력
    input[1].send_keys(pw)

    # 엔터
    input[1].send_keys(Keys.RETURN)
    
    for i in range(0,2):
        #로그인 정보 저장, 알림설정 팝업 제거
        btn_later = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[text()="나중에 하기"]')))
        btn_later.click()
        time.sleep(1)
        
def search(text):
    #검색어에 해시태그 추가
    if text[:1] == '#' :
        url = 'https://www.instagram.com/explore/tags/'+text[1:]
    else :
        url = 'https://www.instagram.com/'+text
    
    # 주소로 이동
    driver.get(url)
    
    # 또 다른 방법. 직접 검색해서 가장 첫번째 값 클릭해서 이동하기
    #hashtagText = text
    #driver.find_element(By.CSS_SELECTOR, '[aria-label="검색"]').click()
    #time.sleep(2)
    #driver.find_element(By.CSS_SELECTOR, '[aria-label="입력 검색"]').send_keys(hashtagText)
    #time.sleep(2)
    #driver.find_element(By.XPATH, f'//*[text()="{hashtagText}"]').click()
    #time.sleep(7)
    
    # ========================================
    # 주의! 여기서부터는 해시태그의 경우를 고려하지 않았음!!
    # 추후 해시태그로 검색 시 보완이 필요할 것으로 보임
    # ========================================
    
     #post 데이터를 저장할 곳
    all_posts_url_list = []
    
    # 얼만큼 가져올지에 따라 다름
    # 로딩이 덜 된 상태가 있을 수 있으므로, 직전값이 아닌 전전값으로 설정한다.
    height_list = []
    
    # 테스트를 위해 임의의 수를 설정하나,
    # 실제 실행시에는 999 값을 넣는 것이 좋음
    for i in range(999):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(3)
        
        posts = driver.find_elements(By.CSS_SELECTOR, '._aabd._aa8k._al3l > a')
        now_posts_url_list = []
        for post in posts :
            now_posts_url_list.append(post.get_attribute("href"))
        
        # 중복 제거하면서 추가
        all_posts_url_list.extend(now_posts_url_list)
        all_posts_url_list = list(set(all_posts_url_list))
        
        # body의 길이 가져오기
        body_height = driver.execute_script('return document.body.scrollHeight;')
        
        if len(height_list) >= 3 :
            if height_list[0] >= body_height :
                print('!! 더이상 스크롤할 곳 없음 !!')
                break;
            else :
                height_list[0] = height_list[1]
                height_list[1] = height_list[2]
                height_list[2] = body_height
        else :
            height_list.append(body_height)

    return all_posts_url_list

def get_post(url):
    
    driver.get(url)
    
    try :
        
        element_text = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '._aacl._aaco._aacu._aacx._aad7._aade')))
        element_date = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.TAG_NAME, 'time')))

        post_text = element_text.text
        post_date = element_date.get_attribute('datetime')[:10]
        
    except :
        print('!! ERROR !! <URL> : '+url)
        post_text = ''
        post_date = ''

    return post_text, post_date

### 1. 포스트 정보 추출

In [ ]:
# 아이디와 비밀번호를 설정.
id = #Instagram 본인 아이디#
pw = #Instagram 본인 패스워드#

In [26]:
# URL 추출 로직
login(id,pw)
main_df = pd.DataFrame(search('popupstorego'), columns = ['url'])
driver.quit()

### 2. 포스트의 게시글 및 게시시간 추출

In [28]:
# Case. 저장한 정보가 없이 처음 실행하는 경우
main_df = main_df[['url']]
main_df['content'] = ''
main_df['date'] = ''

# # Case. 기존 파일 불러오기
# main_df = pd.read_csv('main_df.csv', index_col=0, sep='|')
# main_df = main_df.fillna('')
# main_df

In [30]:
driver = webdriver.Chrome(service=service, options=options)

In [31]:
#apply로 하니깐 중간에 break를 못함..
login(id, pw)

content_bf = 'START' #빈 값만 아니면 아무거나 넣어도 됨

for index, row in tqdm(main_df.loc[main_df['content']==''].iterrows()) :
    content, date = get_post(row['url'])
    
    main_df.loc[index, 'content'] = content
    main_df.loc[index, 'date'] = date
    
    # 이전값, 이번값 모두 빈값이면 break
    if (content_bf == '') and (content == '') :
        break
    else :
        content_bf = content

driver.quit()

24it [01:05,  2.75s/it]


In [32]:
main_df

,url,content,date
0,https://www.instagram.com/p/Crc6oc0uqrB/,다양한 팝업 즐기고 맛집까지 뿌실 수 있는 <성수 풀코스>👀\n\n📝 성수 인기 팝...,2023-04-25
1,https://www.instagram.com/p/CrSg85IPvpx/,"🥄그릭요거트 : YOZM 팝업 카페🥄\n\n슬로우 메이드 그릭요거트 브랜드, YOZ...",2023-04-21
2,https://www.instagram.com/p/CrZ_36LPDqz/,⏰ 팝가가 미리 알려주는 “업커밍 팝업스토어”⏰\n\n📝 포켓몬 스프링 페스타 i...,2023-04-24
3,https://www.instagram.com/p/Cq7iZPOuYNA/,"🌳 지속가능성을 추구하는 스킨케어 브랜드, 클레어스 팝업스토어 🌳\n\n자연, 인간...",2023-04-12
4,https://www.instagram.com/p/Cq4jN5_vF5r/,귀여운 팝업부터 인기 맛집까지 다-모은 <잠실 풀코스>❗\n\n팝업 보고 고기 먹고...,2023-04-11
5,https://www.instagram.com/p/CrAAur5v61Y/,🍨투쿨포스쿨 첫 공식 팝업스토어 : 미스터로댕 젤라또 샵 🍨\n\n투쿨포스쿨이 신제...,2023-04-14
6,https://www.instagram.com/p/CqpFmjtPW8R/,🦕우주먼지 x 마일드무무 팝업스토어🦕\n\n더현대서울에 귀여운 공룡과 오리 인형이 ...,2023-04-05
7,https://www.instagram.com/p/Cqupnm9uI2S/,"🌱디즈니 러브네이처🌱\n\n4월 22일 지구의 날을 맞아,\n건강과 환경을 생각하는...",2023-04-14
8,https://www.instagram.com/p/CrCu2QNPWeR/,💌4월 셋째 주 핫한 팝업 모음. zip💌\n\n✔️도구리 포춘살롱\n📍성동구 서울...,2023-04-15
9,https://www.instagram.com/p/CrR99EqPJrw/,"🚙쏘나타: Into The Edge🚙\n\n현대자동차가 쏘나타 신형, The Edg...",2023-04-21


In [33]:
emojis = ''.join(emoji.EMOJI_DATA.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x): 
    #근데 게시물의 해시태그 데이터에도 중요한 정보가 있을 수 있어서 고려 대상이긴 함..
    x = re.sub(r'#\w+', '', x) #HashTag 데이터 삭제
    x = pattern.sub(' ', x)
    x = emoji.replace_emoji(x, replace='') #emoji 삭제
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

# apply 함수와 lambda 함수를 사용하여 각 행의 'text'에 대해 정규식을 적용
main_df['content'] = main_df['content'].apply(lambda x: clean(x))

In [42]:
main_df['content'][1]

'그릭요거트 : YOZM 팝업 카페 슬로우 메이드 그릭요거트 브랜드, YOZM이 브런치 카페 콘셉트의 팝업스토어를 성수동 오우드에서 운영 중이야- 브랜드 시그니처 컬러, 민트로 꾸며져 있는 공간에선 다양한 맛의 그릭요거트와 함께 카페 오우드와 협업해 개발한 메뉴와 브랜드 굿즈 20여 종을 만나볼 수 있어! 그리스 건축물 느낌이 나는 이국적인 인테리어 덕분에 사진 남기기 딱 좋아~ 방문 인증샷을 SNS에 업로드하면 매주 화,금 당첨자 3명을 추첨해서 그릭요거트 멀티팩 1개를 증정하니 참고해! 그릭요거트를 활용한 다양한 디저트가 맛보고 싶다면? 요즘 팝업 카페로 놀러 가보기- SNS 업로드 시, 매주 화요일 금요일에 당첨자 3명 선정해 그릭요거트 멀티팩 1개 증정 성수동 연무장길 101-1, 오우드 카페 04.14(금)-04.30(일) 매일: 11:00~21:00'

In [36]:
# csv로 저장
main_df.to_csv('main_df.csv', sep='|')

## Address 추출 및 주변정보 검색
이건 아직 미완성

브랜드 NER 추출이 더 시급해보임

In [43]:
main_df = pd.read_csv('main_df.csv', index_col=0, sep='|')

In [44]:
def find_address(text) :
    address_pattern = r'(([가-힣A-Za-z·\d~\-\.]{2,}(로|길)[\d]+)|([가-힣A-Za-z·\d~\-\.]+(읍|동|번지)\s)[\d]+)|([가-힣A-Za-z]+(구)+\s*[가-힣A-Za-z]+(동))|([가-힣a-zA-Z\d]+(아파트|빌라|빌딩|마을))'
    addresses = re.findall(address_pattern, text)
    
    #addresses를 DataFrame에 추가하는 것도 정의할 것!
    
    result_list = []
    if len(addresses) < 1 :
        return result_list
    
    for address in addresses :
        
        if not address[0].strip() : 
            continue
        
        #print(f'!! keyword : {address[0]}')
        
        result_list.append(find_subway(address[0]))

    return result_list

def find_subway(address) :
    api_key = #KAKAO API KEY#
    api = Local(service_key = api_key)
    
    result = api.search_address(address, dataframe=True)
    
    if result.empty :
        return ['해당 주소의 검색 결과 없음']
    
    # 정확하지 않은 지명과 도로명은 제외 (지번 주소, 도로명 주소만!)
    result = result[(result['address_type'] == 'REGION_ADDR') | (result['address_type'] == 'ROAD_ADDR')]
    
    if result.empty :
        return ['정확한 세부 주소까지 입력해야함 : ex. 을지로(X), 을지로 79(O)']
    
    if len(result) > 1 :
        if len(result.drop_duplicates(subset=['x', 'y'])) > 1 :
            return ['같은 주소에 대한 서로 다른 좌표값이 존재함']
    
    addressXy = result.loc[[0]][['x','y']].values[0] #결과가 여러개일 경우, 정확도가 제일 높은 첫 번째 결과로 설정.
    
    # 근처 지하철역 검색
    # 2km 반경으로 지정(추후 수정 가능)
    resultSubway = api.search_category('SW8', x=addressXy[0], y=addressXy[1], radius=2000, dataframe=True)

    if resultSubway.empty :
        return ['해당 주소 2km 반경에 지하철역이 없음']
    
    # 데이터셋 정제
    station_df = resultSubway[['distance', 'place_name']]

    station_df['station'] = station_df.place_name.str.split(' ').str[0]
#     station_df['line'] = station_df.place_name.str.split(' ').str[1]
    station_df.drop(['place_name'], axis='columns', inplace=True)

    station_df['distance'] = station_df['distance'].astype(int)
    
    # 같은 역이면 가장 가까운 거리를 기준으로 그룹화
    # 문제는 호선이 없어지는데.. 애초에 안써도 되긴 함?
    station_df = station_df.groupby('station', as_index=False)['distance'].min()
    
    station_df = station_df.sort_values(by=['distance'], axis=0, ascending=True)
    station_df.reset_index(drop=True, inplace=True)

    # 지하철 정보는 최대 3개만
    station_list = station_df[:3].values.tolist()
    return station_list

In [45]:
main_df['address'] = main_df['content'].apply(find_address)

!! keyword : 아차산로17


C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_df['station'] = station_df.place_name.str.split(' ').str[0]
C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_df.drop(['place_name'], axis='columns', inplace=True)
C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

!! keyword : 아차산로17
!! keyword : 성수이로20


C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_df['station'] = station_df.place_name.str.split(' ').str[0]
C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_df.drop(['place_name'], axis='columns', inplace=True)
C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

!! keyword : 성수이로20


...

C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_df['station'] = station_df.place_name.str.split(' ').str[0]
C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_df.drop(['place_name'], axis='columns', inplace=True)
C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

!! keyword : 한강대로23
!! keyword : 선릉로155
!! keyword : 성수이로7
!! keyword : 한강대로23


C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_df['station'] = station_df.place_name.str.split(' ').str[0]
C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_df.drop(['place_name'], axis='columns', inplace=True)
C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

!! keyword : 선릉로155
!! keyword : 성수이로7


C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_df['station'] = station_df.place_name.str.split(' ').str[0]
C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_df.drop(['place_name'], axis='columns', inplace=True)
C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [47]:
main_df['address']

0                                                    []
1                                                    []
2                                                    []
3              [[[뚝섬역, 115], [서울숲역, 501], [한양대역, 895]]]
4                                                    []
5                                                    []
6                                                    []
7                                                    []
8              [[[뚝섬역, 115], [서울숲역, 501], [한양대역, 895]]]
9          [[[성수역, 856], [서울숲역, 1115], [뚝섬유원지역, 1264]]]
10                                                   []
11         [[[성수역, 856], [서울숲역, 1115], [뚝섬유원지역, 1264]]]
12    [[[용산역, 693], [신용산역, 805], [이촌역, 1027]], [[성수역...
13             [[[용산역, 693], [신용산역, 805], [이촌역, 1027]]]
14                                                   []
15    [[해당 주소의 검색 결과 없음], [[성수역, 974], [서울숲역, 1141],...
16                                                   []
17                                              

In [48]:
pd.DataFrame(find_subway('동교로29'), columns=['station', 'distance(m)'])

C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_df['station'] = station_df.place_name.str.split(' ').str[0]
C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_df.drop(['place_name'], axis='columns', inplace=True)
C:\Users\david\AppData\Local\Temp\ipykernel_29608\3760548337.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

,station,distance(m)
0,망원역,561
1,합정역,827
2,마포구청역,1239
